In [61]:
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler

In [62]:
df = pd.read_csv('https://raw.githubusercontent.com/murpi/wilddata/master/quests/spotify.zip')

In [63]:
def popularity_score(popularity):
  popularity = popularity / 20
  popularity = math.ceil(popularity)
  return popularity

In [64]:
df.drop('time_signature', axis = 1, inplace = True)
df = df[(df['genre'] != 'Soundtrack') & (df['genre'] != 'Movie')]
df = df[(df['duration_ms'] >= 90000) & (df['duration_ms'] <= 900000)]
df['genre'].replace("Children’s Music", "Children's Music", inplace = True)
top_songs = df[df['popularity'] >= 80]
top_scaled = top_songs.copy()
scaler = MinMaxScaler()
top_scaled['loudness_scaled'] = scaler.fit_transform(top_songs['loudness'].values.reshape(-1, 1))

In [65]:
top_scaled.to_csv('data/top_polar.csv.zip', compression='zip', index=False)

In [66]:
df_music_dummies = pd.concat([df , df['genre'].str.get_dummies()], axis = 1)
df_music_dummies['mode'] = df_music_dummies['mode'].factorize()[0]
df_music_dummies["popularity_score"] = df["popularity"].apply(popularity_score)
cols = ['genre', 'key', 'popularity']
df_music_dummies.drop(cols, axis=1, inplace=True)
df_music_without_0_pop = df_music_dummies[df_music_dummies['popularity_score'] != 0]

In [67]:

df_final = df_music_without_0_pop.copy()
df_final['tempo'].apply(round)
df_final['duration_ms'] = round(df_final['duration_ms'] / 1000)
df_final = df_final.astype({'tempo': 'int', 'duration_ms': 'int'})
df_final.drop_duplicates(subset = ['track_id'], inplace = True)

In [74]:
df_final.to_csv('data/music_ml.csv.zip', compression='zip', index = False)

In [69]:
popularity_genre = round(df.groupby(by=['genre']).mean('popularity'),2).sort_values(['popularity'], ascending=False).reset_index()

In [70]:
popularity_genre.to_csv('data/pop_genre.csv.zip', compression='zip', index=False)

In [71]:
df_test = df_final['track_name'].value_counts()
df_test = pd.DataFrame(df_test)
df_test

,track_name
Home,93
You,69
Stay,61
Closer,58
Wake Up,55
...,...
"Die Walküre, WWV 86B: Leb wohl, du kühnes, herrliches Kind - Live",1
"Carmen, Act IV, No.27 Duo et Choeur final: Tu m'aimes donc plus (Don José/Carmen/Choeurs)",1
Carmen Suite No. 2: La Garde montante. Allegro (Act I),1
J.S. Bach: Ave Maria,1


In [75]:
df_final[df_final['track_name'] == 'Poker Face']

,artist_name,track_name,track_id,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,...,Pop,R&B,Rap,Reggae,Reggaeton,Rock,Ska,Soul,World,popularity_score
13645,Lady Gaga,Poker Face,0WfKDYeUAoLA3vdvLKKWMW,0.139000,0.846,237,0.815,0.000002,0.131,-4.512,...,0,0,0,0,0,0,0,0,0,4
31708,yuchaP,Poker Face,6PP3V8WppdOquSc7o0zmMF,0.000971,0.516,200,0.893,0.000106,0.140,-3.836,...,0,0,0,0,0,0,0,0,0,1


In [76]:
df_final.columns

Index(['artist_name', 'track_name', 'track_id', 'acousticness', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness',
       'mode', 'speechiness', 'tempo', 'valence', 'A Capella', 'Alternative',
       'Anime', 'Blues', 'Children's Music', 'Classical', 'Comedy', 'Country',
       'Dance', 'Electronic', 'Folk', 'Hip-Hop', 'Indie', 'Jazz', 'Opera',
       'Pop', 'R&B', 'Rap', 'Reggae', 'Reggaeton', 'Rock', 'Ska', 'Soul',
       'World', 'popularity_score'],
      dtype='object')